In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="6"

In [2]:
import os
import gc

import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt
import h5py
# For data manipulation
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

from sklearn.metrics import roc_curve, auc, roc_auc_score
# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold, GroupKFold
#from timm.data.auto_augment import auto_augment_transform
from torchvision import datasets, transforms
import albumentations as A
# For Image Models
import timm
from glob import glob
# Albumentations for augmentations
#import albumentations as A
#from albumentations.pytorch import ToTensorV2
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy
from timm.utils import ModelEmaV2
from libauc.sampler import DualSampler  # data resampling (for binary class)
from models import ClassifierNet
import cv2

from apex import amp
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

ERROR:albumentations.check_version:Error fetching version info
Traceback (most recent call last):
  File "/data/guanghan/miniconda3/envs/kaggle/lib/python3.10/urllib/request.py", line 1348, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/data/guanghan/miniconda3/envs/kaggle/lib/python3.10/http/client.py", line 1283, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/data/guanghan/miniconda3/envs/kaggle/lib/python3.10/http/client.py", line 1329, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/data/guanghan/miniconda3/envs/kaggle/lib/python3.10/http/client.py", line 1278, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/data/guanghan/miniconda3/envs/kaggle/lib/python3.10/http/client.py", line 1038, in _send_output
    self.send(msg)
  File "/data/guanghan/miniconda3/envs/kaggle/lib/python3.10/http/client.py", line 976, in send
    self.conne

In [3]:
import wandb

# 初始化 Weights & Biases
wandb.init(project="isic_2024",)


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: whhhhh. Use `wandb login --relogin` to force relogin


In [4]:
CONFIG = {
    "seed": 42,
    "epochs": 10,
    "img_size": 384,
    "model_name": "tf_efficientnetv2_l.in21k",
    "checkpoint_path" : "./resnet18_pretrained/fold_0_AUROC0.1374_Loss0.1938_epoch8.bin",
    'output_dir': 'resnet18_exp1',
    "train_batch_size": 32,
    "valid_batch_size": 32,
    "learning_rate": 5e-4,
    "scheduler": "CosineAnnealingLR",
    "min_lr": 1e-7,
    "T_max": 500,
    "weight_decay": 0,
    "fold" : 0,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "undersample_rate": 20,
    'using_mix_up':False
}

In [5]:
os.makedirs(CONFIG['output_dir'], exist_ok=True)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Set Seed for Reproducibility</h1></span>

In [6]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [7]:
df_2024 = pd.read_csv(f"./2024/train-metadata.csv")
df_2024['file_path'] = df_2024['isic_id'].apply(lambda x:f'./2024/train-image/image/{x}.jpg')
#df_2024 = df_2024[['patient_id', 'isic_id', 'target', 'file_path']]

In [8]:
df_2020 = pd.read_csv("./2020/train-metadata.csv")
df_2020['file_path'] = df_2020['isic_id'].apply(lambda x:f'./2020/train-image/image/{x}.jpg')
df_2020 = df_2020[['patient_id', 'isic_id', 'target', 'file_path']]

df_2019 = pd.read_csv("./2019/train-metadata.csv")
df_2019['file_path'] = df_2019['isic_id'].apply(lambda x:f'./2019/train-image/image/{x}.jpg')
df_2019 = df_2019[['patient_id', 'isic_id', 'target', 'file_path']]

df_2018 = pd.read_csv("./2018/train-metadata.csv")
images_path = glob('./2018/train-image/image/*.jpg')
df_2018['file_path'] = df_2018['isic_id'].apply(lambda x:f'./2018/train-image/image/{x}.jpg')
df_2018 = df_2018[['patient_id', 'isic_id', 'target', 'file_path']]
df_2018 = df_2018[df_2018['file_path'].isin(images_path)]

In [9]:
df = df_2024 #pd.concat([df_2020, df_2019, df_2018]).reset_index(drop=True)

In [10]:
num_cols   = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'lesion_size_ratio',                 # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',                # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                      # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',                # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',           # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'border_complexity',                 # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',                  # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',              # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',           # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',           # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',           # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',       # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',       # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',                 # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',                 # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',              # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',         # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',             # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',            # border_complexity       + lesion_shape_index
    'color_contrast_index',              # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',                   # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',            # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',               # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',                  # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',       # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',             # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',          # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',    # tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',        # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',              # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',          # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',         # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',    # tbp_lv_nevi_confidence  / age_approx
    #'age_normalized_nevi_confdnce_2',
    'color_asymmetry_index',             # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',           # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                       # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',           # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',               # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',           # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',           # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
    
]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']

label_encoders=  {}
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1)) #StandardScaler()
label_encoder = LabelEncoder()
for column in cat_cols:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le
# 进行编码

df =  df_2024[['patient_id', 'isic_id', 'target', 'file_path']+num_cols+cat_cols]
df[num_cols] = scaler.fit_transform(df[num_cols])
df

,patient_id,isic_id,target,file_path,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,...,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,sex,anatom_site_general,tbp_tile_type,tbp_lv_location,tbp_lv_location_simple,attribution
0,IP_1235828,ISIC_0015670,0,./2024/train-image/image/ISIC_0015670.jpg,0.375,-0.851095,-0.102881,0.099400,0.004887,-0.247862,...,-0.028571,-0.286449,0.133118,-0.183826,1,2,1,12,4,4
1,IP_8170065,ISIC_0015845,0,./2024/train-image/image/ISIC_0015845.jpg,0.375,-0.992701,0.349720,0.494297,-0.016605,-0.217479,...,-0.371429,0.008264,0.787694,0.142046,1,1,1,0,0,4
2,IP_6724798,ISIC_0015864,0,./2024/train-image/image/ISIC_0015864.jpg,0.375,-0.824818,-0.010870,0.136977,0.406357,0.239195,...,0.200000,0.207932,0.717163,0.717002,1,3,0,16,5,4
3,IP_4111386,ISIC_0015902,0,./2024/train-image/image/ISIC_0015902.jpg,0.500,-0.837956,-0.339758,-0.078346,-0.194041,-0.392661,...,0.485714,-0.219189,0.696873,0.145924,1,0,0,19,6,0
4,IP_8313778,ISIC_0024200,0,./2024/train-image/image/ISIC_0024200.jpg,0.250,-0.873723,0.073970,0.264057,-0.011739,-0.158138,...,-0.771429,-0.108309,0.728530,0.025096,1,0,1,19,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,IP_1140263,ISIC_9999937,0,./2024/train-image/image/ISIC_9999937.jpg,0.625,-0.576642,-0.010929,0.042252,0.031807,-0.104138,...,0.142857,0.245916,0.520086,0.356273,1,0,0,19,6,1
401055,IP_5678181,ISIC_9999951,0,./2024/train-image/image/ISIC_9999951.jpg,0.375,-0.845985,-0.113409,0.089201,0.267848,0.112787,...,-0.714286,0.092871,0.663540,0.371353,1,3,1,16,5,4
401056,IP_0076153,ISIC_9999960,0,./2024/train-image/image/ISIC_9999960.jpg,0.500,-0.923358,-0.217799,-0.069685,0.111101,-0.117786,...,-0.542857,0.096538,0.440455,-0.394544,0,0,0,19,6,3
401057,IP_5231513,ISIC_9999964,0,./2024/train-image/image/ISIC_9999964.jpg,-0.375,-0.868613,-0.022207,-0.191141,0.059713,-0.091611,...,0.600000,-0.007523,0.312739,-0.296878,0,0,0,18,6,5


In [11]:
cat_dim = [df[i].max()+1 for i in cat_cols]
cat_dim

[3, 6, 2, 21, 8, 7]

In [12]:
len(df[df['target']==1])

393

In [13]:
len(df[df['target']==0])

400666

In [14]:
CONFIG['T_max'] = df.shape[0] * (CONFIG["n_fold"]-1) * CONFIG['epochs'] // CONFIG['train_batch_size'] // CONFIG["n_fold"]
CONFIG['T_max']

100264

In [15]:
df[num_cols] = df[num_cols].fillna(-1)
inputs = df[num_cols].values
if np.isnan(inputs).any() or np.isinf(inputs).any():
    print("Input data contains NaN or Inf values!")

In [16]:
sgkf = StratifiedGroupKFold(n_splits=CONFIG['n_fold'])

for fold, ( _, val_) in enumerate(sgkf.split(df, df.target,df.patient_id)):
      df.loc[val_ , "kfold"] = int(fold)

In [17]:
def make_valid_augment(image_size):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        #transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])
    # transform = A.Compose([
    #     A.Resize(image_size, image_size),
    #     A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    # ])
    return transform




def make_train_augment(image_size):
    augment_policy = 'autoaugment'  # 可以替换为其他策略，如 'autoaugment', 'trivialaugment'

    # 创建数据增强变换
    transform = transforms.Compose([
        transforms.Resize(image_size),
        #transforms.RandomCrop((224, 224)),
        #transforms.RandomHorizontalFlip(),
        #transforms.Resize(image_size),
        #auto_augment_transform(config_str = 'original' ,hparams = {'translate_const': 100,}),
        transforms.ToTensor(),
        #transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
     ])
    # transform = A.Compose([
    #     A.Transpose(p=0.5),
    #     A.VerticalFlip(p=0.5),
    #     A.HorizontalFlip(p=0.5),
    #     A.RandomBrightnessContrast(p=0.75),
    #     A.OneOf([
    #         A.MotionBlur(blur_limit=5),
    #         A.MedianBlur(blur_limit=5),
    #         A.GaussianBlur(blur_limit=5),
    #         A.GaussNoise(var_limit=(5.0, 30.0)),
    #     ], p=0.7),

    #     A.OneOf([
    #         A.OpticalDistortion(distort_limit=1.0),
    #         A.GridDistortion(num_steps=5, distort_limit=1.),
    #         A.ElasticTransform(alpha=3),
    #     ], p=0.7),

    #     A.CLAHE(clip_limit=4.0, p=0.7),
    #     A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    #     A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    #     A.Resize(image_size, image_size),
    #     A.CoarseDropout(max_height=int(image_size * 0.375), max_width=int(image_size * 0.375), p=0.7),    
    #     A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    # ])
    return transform


class SkinDataset(Dataset):
    def __init__(self, df, augment):
        self.df = df
        self.targets = df['target'].tolist()
        self.augment = augment
        self.length = len(self.df)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        d = self.df.iloc[index]
        img_path = d.file_path
        m = Image.open(img_path)
        r = self.augment(m)
        

        r = {
            #'index': index,
            'image': r,
            'target':self.targets[index]
        }
        return r
class MultiModSkinDataset(Dataset):
    def __init__(self, df, augment):
        self.df = df.reset_index(drop=True)
        self.targets = df['target'].tolist()
        self.file_path = df['file_path'].tolist()
        self.augment = augment
        self.length = len(self.df)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        
        img_path = self.file_path[index]
        # false_color = cv2.imread(img_path)
        # false_color = cv2.cvtColor(false_color, cv2.COLOR_BGR2RGB)
        # transformed = self.augment(image=false_color)
        # transformed_image = transformed['image']
        # r = torch.from_numpy(transformed_image).permute(2, 0, 1)
        m = Image.open(img_path)
        r = self.augment(m)
        num_feature =  np.array(self.df.loc[index, num_cols].values, dtype=np.float32)
        cat_feature = np.array(self.df.loc[index, cat_cols].values, dtype=np.int64)
        r = {
            #'index': index,
            'image': r,
            'target':self.targets[index],
            'num_cols': num_feature,
            'cat_cols': cat_feature,
        }
        return r

def null_collate(batch):
    d = {}
    key = batch[0].keys()
    for k in key:
        d[k] = [b[k] for b in batch]
    d['image'] = torch.stack(d['image']).float()
    d['target'] = torch.as_tensor(d['target'], dtype=torch.long)#.float()

    return d

def multi_mod_null_collate(batch):
    d = {}
    key = batch[0].keys()
    for k in key:
        d[k] = [b[k] for b in batch]
    #print(d['num_cols'])
    #print(d['cat_cols'])
    d['image'] = torch.stack(d['image']).float()
    d['num_cols'] = torch.stack([torch.as_tensor(i) for i in d['num_cols']])
    d['cat_cols'] = torch.stack([torch.as_tensor(i) for i in d['cat_cols']]).long()
    d['target'] = torch.as_tensor(d['target'], dtype=torch.long)#.float()

    return d

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Augmentations</h1></span>

In [18]:
def F_rgb2hsv(rgb: torch.Tensor) -> torch.Tensor:
    cmax, cmax_idx = torch.max(rgb, dim=1, keepdim=True)
    cmin = torch.min(rgb, dim=1, keepdim=True)[0]
    delta = cmax - cmin
    hsv_h = torch.empty_like(rgb[:, 0:1, :, :])
    cmax_idx[delta == 0] = 3
    hsv_h[cmax_idx == 0] = (((rgb[:, 1:2] - rgb[:, 2:3]) / delta) % 6)[cmax_idx == 0]
    hsv_h[cmax_idx == 1] = (((rgb[:, 2:3] - rgb[:, 0:1]) / delta) + 2)[cmax_idx == 1]
    hsv_h[cmax_idx == 2] = (((rgb[:, 0:1] - rgb[:, 1:2]) / delta) + 4)[cmax_idx == 2]
    hsv_h[cmax_idx == 3] = 0.
    hsv_h /= 6.
    hsv_s = torch.where(cmax == 0, torch.tensor(0.).type_as(rgb), delta / cmax)
    hsv_v = cmax
    return torch.cat([hsv_h, hsv_s, hsv_v], dim=1)


In [19]:
class Net(nn.Module):
    def __init__(self, pretrained=False):
        super(Net, self).__init__()
        self.output_type = ['infer', 'loss']
        arch = CONFIG['model_name']

        self.model_name = CONFIG['model_name']

        self.model = timm.create_model(model_name=arch, pretrained=pretrained, in_chans=6,  num_classes=0, global_pool='')
        #print(f'Feature channels: {self.model.feature_info}')
        in_features =  self.model.num_features
        self.target=nn.Linear(in_features, 2)

        self.dropout = nn.ModuleList([
            nn.Dropout(0.5) for i in range(5)
        ]) #droupout augmentation

       
    def forward(self, batch):
        image = batch['image']
        batch_size = len(image)
        image = image.float()/255
        image = torch.cat([image, F_rgb2hsv(image)],1)
        # image = (image-self.mean)/self.std
        x = self.model(image)
        pool = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)

        if self.training:
            logit=0
            for i in range(len(self.dropout)):
                logit += self.target(self.dropout[i](pool))
            logit = logit/len(self.dropout)
        else:
            logit = self.target(pool)
        output = {}
        target = batch['target']
        
        #output['loss'] = self.loss_fn(logit, target)
        pred_prob = logit.softmax(dim=-1)[:, 1]
        #print(pred_prob)
        output['logit'] = logit
        output['pred'] = pred_prob
        output['pool'] = pool
        return output


In [20]:
from tab_transformer_pytorch import TabTransformer
sigmoid = nn.Sigmoid()


class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * sigmoid(i)
        ctx.save_for_backward(i)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))


class Swish_Module(nn.Module):
    def forward(self, x):
        return Swish.apply(x)

class MultiModNet(nn.Module):
    def __init__(self, pretrained=False):
        super(MultiModNet, self).__init__()
        self.output_type = ['infer', 'loss']
        arch = CONFIG['model_name']

        self.model_name = CONFIG['model_name']

        self.model = timm.create_model(model_name=arch, pretrained=pretrained, in_chans=6,  num_classes=0, global_pool='')
        self.meta = nn.Sequential(
                nn.Linear( len(cat_cols)+len(num_cols), 512),
                nn.BatchNorm1d(512),
                Swish_Module(),
                nn.Dropout(p=0.3),
                nn.Linear(512, 128),
                nn.BatchNorm1d(128),
                Swish_Module(),
            )
        self.mlp = nn.Linear(self.model.num_features + 128, 2)
    def forward(self, batch):
        image = batch['image']
        num_cols = batch['num_cols']
        cat_cols = batch['cat_cols']
        #print(cat_cols)
        #print(num_cols)
        batch_size = len(image)
        image = image.float()/255
        image = torch.cat([image, F_rgb2hsv(image)],1)
        # image = (image-self.mean)/self.std
        x = self.model(image)
        pool = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        
        meta_feature = torch.cat([cat_cols, num_cols], dim=1)
        meta_feature = self.meta(meta_feature)

        logit = torch.cat((meta_feature, pool), dim=1)
        logit = self.mlp(logit)
        output = {}
        target = batch['target']
        
        #output['loss'] = self.loss_fn(logit, target)
        pred_prob = logit.softmax(dim=-1)[:, 1]
        #print(pred_prob)
        output['logit'] = logit
        output['pred'] = pred_prob
        output['pool'] = pool
        return output


In [21]:
from tab_transformer_pytorch import TabTransformer
sigmoid = nn.Sigmoid()


class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * sigmoid(i)
        ctx.save_for_backward(i)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))


class Swish_Module(nn.Module):
    def forward(self, x):
        return Swish.apply(x)

class MultiModTabNet(nn.Module):
    def __init__(self, pretrained=False):
        super(MultiModTabNet, self).__init__()
        self.output_type = ['infer', 'loss']
        arch = CONFIG['model_name']

        self.model_name = CONFIG['model_name']

        self.model = timm.create_model(model_name=arch, pretrained=pretrained, in_chans=6,  num_classes=0, global_pool='')
        self.tab_transformer = TabTransformer(
                                categories = cat_dim,      # tuple containing the number of unique values within each category
                                num_continuous = self.model.num_features,# + len(num_cols),                # number of continuous values
                                dim = 32,                           # dimension, paper set at 32
                                dim_out = 2,                        # binary prediction, but could be anything
                                depth = 6,                          # depth, paper recommended 6
                                heads = 8,                          # heads, paper recommends 8
                                attn_dropout = 0.1,                 # post-attention dropout
                                ff_dropout = 0.1,                   # feed forward dropout
                                mlp_hidden_mults = (4, 2),          # relative multiples of each hidden dimension of the last mlp to logits
                                mlp_act = nn.Tanh(),                # activation for final mlp, defaults to relu, but could be anything else (selu etc)
                                continuous_mean_std = None # (optional) - normalize the continuous values before layer norm
                            )
        
    def forward(self, batch):
        image = batch['image']
        num_cols = batch['num_cols']
        cat_cols = batch['cat_cols']
        #print(cat_cols)
        #print(num_cols)
        batch_size = len(image)
        image = image.float()/255
        image = torch.cat([image, F_rgb2hsv(image)],1)
        # image = (image-self.mean)/self.std
        x = self.model(image)
        pool = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        
        num_feature = pool#torch.cat([pool, num_cols], dim=1)
        

        logit =self.tab_transformer(cat_cols, num_feature)
        
        output = {}
        target = batch['target']
        
        #output['loss'] = self.loss_fn(logit, target)
        pred_prob = logit.softmax(dim=-1)[:, 1]
        #print(pred_prob)
        output['logit'] = logit
        output['pred'] = pred_prob
        output['pool'] = pool
        return output


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Training Function</h1></span>

In [22]:
def comp_score(gts, preds, min_tpr: float=0.80):
    v_gt = abs(np.asarray(gts)-1)
    v_pred = np.array([1.0 - x for x in preds])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [23]:
def train_one_epoch(model, model_ema, optimizer, scheduler, dataloader, device, epoch, mixup_fn=None):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0
    if CONFIG['using_mix_up']:
        loss_fn = SoftTargetCrossEntropy().cuda()
    else:
        loss_fn = F.cross_entropy
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    preds = []
    gts = []
    for step, data in bar:
        for k, v in data.items():
            data[k] = v.to(device)
        
        batch_size = data['image'].size(0)
        if CONFIG['using_mix_up']:
            image = data['image']
            ori_target = data['target']
            
            image, target = mixup_fn(image, ori_target)
            data['image'] = image
            data['target'] = target

        outputs = model(data)
        if CONFIG['using_mix_up']:
            loss = loss_fn(outputs['logit'], data['target'])
            targets = ori_target
        else:
            loss = loss_fn(outputs['logit'], data['target'].long())
            targets = data['target'].long()
        pred = outputs['pred']
        #print(outputs['logit'])
        # for name, param in model.named_parameters():
        #     if torch.isnan(param).any() or torch.isinf(param).any():
        #         print(f"Parameter {name} contains NaN or Inf values!")

        gts.append(targets.squeeze().cpu().numpy().reshape(-1))
        preds.append(pred.squeeze().detach().cpu().numpy().reshape(-1))
        loss = loss / CONFIG['n_accumulate']
            
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            wandb.log({"epoch": epoch, "loss": loss.item(), })
            grad = torch.nn.utils.clip_grad_norm_(model.parameters(), 500)
            #print(grad.item())
            #print(loss.item())
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
            #model_ema.update(model)
        #auroc = comp_score(targets.squeeze().cpu().numpy(), preds.squeeze().detach().cpu().numpy())
        
        running_loss += (loss.item() * batch_size)
        #running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        #epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Grad = grad.item())
    gc.collect()
    gts = np.concatenate(gts)
    preds = np.concatenate(preds)
    epoch_auroc = comp_score(gts, preds)
    return epoch_loss, epoch_auroc

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Validation Function</h1></span>

In [24]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    preds = []
    gts = []
    pools = []
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        for k, v in data.items():
            data[k] = v.to(device)
        
        batch_size = data['image'].size(0)
        
        outputs = model(data)
        loss = F.cross_entropy(outputs['logit'],  data['target'].long())
        pred = outputs['pred']
        targets = data['target']
        pool = outputs['pool']
        gts.append(targets.squeeze().cpu().numpy().reshape(-1))
        preds.append(pred.squeeze().detach().cpu().numpy().reshape(-1))
        pools.append(pool.squeeze().detach().cpu().numpy())
        #auroc = comp_score(targets.squeeze().cpu().numpy(), preds.squeeze().detach().cpu().numpy())
        running_loss += (loss.item() * batch_size)
        #running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        #epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss)   
    
    gc.collect()
    gts = np.concatenate(gts)
    preds = np.concatenate(preds)
    epoch_auroc = comp_score(gts, preds)
    pools = np.vstack(pools)
    return epoch_loss, epoch_auroc, pools, preds

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Run Training</h1></span>

In [25]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    history = defaultdict(list)
    mixup_args = {
                'mixup_alpha': 0,
                'cutmix_alpha': 1,
                'cutmix_minmax': None,
                'prob': 1.0,
                'switch_prob': 0,
                'mode': 'elem',
                'label_smoothing': 0,
                'num_classes': 2}
    mixup_fn = Mixup(**mixup_args)
    if not CONFIG['using_mix_up']:
        mixup_fn = None
    model_ema = ModelEmaV2(model, decay=0.9999)
    wandb.watch(model, log="all", log_freq=10)

    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_loader, valid_loader = prepare_loaders_mulit_mod(df, fold=CONFIG["fold"])
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, model_ema, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch, mixup_fn=mixup_fn)
        
        val_epoch_loss, val_epoch_auroc, val_features, val_preds = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train AUROC'].append(train_epoch_auroc)
        history['Valid AUROC'].append(val_epoch_auroc)
        #history['lr'].append( scheduler.get_lr()[0] )
        wandb.log({"val_loss": val_epoch_loss, "val_p_auc": val_epoch_auroc})
        # deep copy the model
        if best_epoch_auroc <= val_epoch_auroc:
            print(f"Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
            best_epoch_auroc = val_epoch_auroc
            PATH = f"best_{best_epoch_auroc}.bin"
            output_dir = CONFIG['output_dir']
            torch.save(model.state_dict(), f'./{output_dir}/fold_{fold}_{PATH}')
            # Save a model file from the current directory
            print(f"Model Saved")
            best_val_features = val_features
            best_val_preds = val_preds
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best AUROC: {:.4f}".format(best_epoch_auroc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history, best_val_features, best_val_preds

In [26]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [27]:
def prepare_loaders_mulit_mod(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    # df_train_pos = df_train[df_train['target']==1]
    # df_train_neg = df_train[df_train['target']==0].smaple(n=CONFIG['undersample_rate']*len(df_train_pos), random_state=64)
    # df_train = pd.concat([df_train_pos, df_train_neg]).sample(frac=1).reset_index(drop=True)

    
    train_dataset = MultiModSkinDataset(df_train, augment=make_train_augment(image_size=CONFIG['img_size']))
    valid_dataset = MultiModSkinDataset(df_valid, augment=make_valid_augment(image_size=CONFIG['img_size']))
    sampler = DualSampler(train_dataset, batch_size=CONFIG['train_batch_size'], sampling_rate=0.05)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=1, shuffle=True, pin_memory=True, collate_fn=multi_mod_null_collate,
                              #sampler=sampler
                             )
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=1, shuffle=False, pin_memory=True, collate_fn=multi_mod_null_collate,)
    
    return train_loader, valid_loader

In [28]:
oof = []
for i in range(1):
    CONFIG["fold"] = i
    
    model = MultiModNet(False)
    pre_weight = torch.load(CONFIG['checkpoint_path'])
    #model.load_state_dict(pre_weight, strict=False)
    model.cuda()
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], 
                           weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    model, history, best_val_features, best_val_preds = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=CONFIG["fold"]
                                 )
    
    val_df = df[df.kfold == fold].reset_index(drop=True)[['patient_id', 'isic_id']]
    val_features =  pd.DataFrame(best_val_features, columns=[f'cnn_features_{i}' for i in range(best_val_features.shape[1])])
    val_df = pd.concat([val_df, val_features], axis=1)
    val_df['preds'] = best_val_preds
    oof.append(val_df)
oof = pd.concat(oof)

[INFO] Using GPU: NVIDIA GeForce RTX 4090



  0%|          | 0/10027 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 

In [29]:
oof = []
for i in range(5):
    CONFIG["fold"] = i
    
    model = MultiModNet(False)
    pre_weight = torch.load(CONFIG['checkpoint_path'])
    model.load_state_dict(pre_weight, strict=False)
    model.cuda()
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], 
                           weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    model, history, best_val_features, best_val_preds = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=CONFIG["fold"]
                                 )
    
    val_df = df[df.kfold == fold].reset_index(drop=True)[['patient_id', 'isic_id']]
    val_features =  pd.DataFrame(best_val_features, columns=[f'cnn_features_{i}' for i in range(best_val_features.shape[1])])
    val_df = pd.concat([val_df, val_features], axis=1)
    val_df['preds'] = best_val_preds
    oof.append(val_df)
oof = pd.concat(oof)

[INFO] Using GPU: NVIDIA GeForce RTX 4090



100%|██████████| 314/314 [02:28<00:00,  2.12it/s, Epoch=1, Valid_Loss=0.00669]


Validation AUROC Improved (-inf ---> 0.13087291449160318)
Model Saved



100%|██████████| 314/314 [02:31<00:00,  2.08it/s, Epoch=2, Valid_Loss=0.00593]


Validation AUROC Improved (0.13087291449160318 ---> 0.15082315044378586)
Model Saved



...

Validation AUROC Improved (0.16106171513364456 ---> 0.16504186656182065)
Model Saved



100%|██████████| 314/314 [02:29<00:00,  2.09it/s, Epoch=8, Valid_Loss=0.00533]


Validation AUROC Improved (0.16504186656182065 ---> 0.16700409930249946)
Model Saved



 16%|█▌        | 197/1254 [01:36<08:47,  2.00it/s, Epoch=9, Grad=0.0173, Train_Loss=0.00407] ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 16%|█▌        | 198/1254 [02:54<15:31,  1.13it/s, Epoch=9, Grad=0.00909, Train_Loss=0.00406]


RuntimeError: DataLoader worker (pid(s) 15903, 15904) exited unexpectedly

In [ ]:
# sgkf
# baseline fold0  0.1363120993115752
# + mix-up fold0 0.13397867014309467 
# + pretrained fold0 0.14820244272661473
# + pretrained + autoaug fold0 0.144
# + pretrained + ema fold0 0.09
# + pretrained + meta data fold0  0.16700409930249946
# meta data fold0  0.162

In [ ]:
gts = []
for i in range(5):
    gts.append(df[df.kfold == i]['target'])
gts = np.concatenate(gts)

In [ ]:
gts

In [ ]:
comp_score(gts, oof['preds'].values)

In [ ]:
comp_score

In [ ]:
oof.to_csv('./'+CONFIG['output_dir']+'/oof.csv', index=None)

In [ ]:
history = pd.DataFrame.from_dict(history)
history.to_csv("history.csv", index=False)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Logs</h1></span>

In [ ]:
plt.plot( range(history.shape[0]), history["Train Loss"].values, label="Train Loss")
plt.plot( range(history.shape[0]), history["Valid Loss"].values, label="Valid Loss")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.plot( range(history.shape[0]), history["Train AUROC"].values, label="Train AUROC")
plt.plot( range(history.shape[0]), history["Valid AUROC"].values, label="Valid AUROC")
plt.xlabel("epochs")
plt.ylabel("AUROC")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# plt.plot( range(history.shape[0]), history["lr"].values, label="lr")
# plt.xlabel("epochs")
# plt.ylabel("lr")
# plt.grid()
# plt.legend()
# plt.show()